<a href="https://colab.research.google.com/github/a123700/Leo/blob/master/OCW_Recommendation_and_Unsupervised_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%reset -f

#Load Data

In [ ]:
api_token = {"username":"srhgccc","key":"41c36ccc1ee7bc07f70e459ccbf7df7c"}
import json
import zipfile
import os
 
if not os.path.exists("/root/.kaggle"):
    os.makedirs("/root/.kaggle")
 
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)
!chmod 600 /root/.kaggle/kaggle.json
 
if not os.path.exists("/kaggle"):
    os.makedirs("/kaggle")
os.chdir('/kaggle')

In [ ]:
!kaggle datasets download -d srhgccc/ocw-dataset

In [ ]:
!unzip ocw-dataset.zip

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
dataset = pd.read_csv('NewOCW.csv')
dataset = dataset.dropna()
dataset['client_id'] = dataset['client_id'].astype('str')
dataset['date'] = pd.to_datetime(dataset['date'])
dateset = dataset.sort_values(by='date', ascending=True, inplace=True)
dataset = dataset.drop(columns=['page', 'links'])

In [ ]:
print(dataset.shape)

In [ ]:
import time
from datetime import datetime

dataset['time_delta'] = (max(dataset['date']) - dataset['date'])
dataset['time_delta'] = list(map(lambda x: x.days, dataset['time_delta']))

In [ ]:
r = 0.75
dataset['user_score'] = r**dataset['time_delta']

In [ ]:
dataset.head(2)

In [ ]:
user_mappings = {k:v for v, k in enumerate(dataset['client_id'].unique())}
dataset['client_id'] = dataset['client_id'].map(user_mappings)

#Data Preprocessing

In [ ]:
dataset['hour'] = list(map(lambda x: str(x)[11:13], dataset['date']))
cc = dataset.groupby('client_id')['title'].nunique()
cc = pd.DataFrame(cc)
cc = cc.rename(columns={'title': 'number'})
dataset = pd.merge(dataset, cc, on="client_id")

In [ ]:
dataset.head(2)

# Input Output Split

In [ ]:
# Use previous x-n behaviors to predict last n behaviors
from tqdm import tqdm

X = pd.DataFrame()
Y = pd.DataFrame()
n = 1
person = []
for i in tqdm(range(max(dataset['client_id'])+1)):
  temp = dataset[dataset['client_id'] == i]
  if temp.shape[0] < n+1:
    person.append(i)
    continue
  X = pd.concat([X, temp.iloc[:-n, :]], axis = 0)  #rbind
  Y = pd.concat([Y, temp.iloc[-n:, :]], axis = 0)  
del temp

In [ ]:
print(X.shape)
print(Y.shape)

# Train Test Split

In [ ]:
from random import sample
import random

random.seed(1234)

id = list(X['client_id'].unique())
id_train = sample(list(id), int(len(id)*0.75))
id_test = []
for i in id:
  if i not in id_train:
    id_test.append(i)

In [ ]:
print(len(id_train))
print(len(id_test))

In [ ]:
x_train = X[X['client_id'].isin(id_train)]
x_test = X[X['client_id'].isin(id_test)]

y_train = Y[Y['client_id'].isin(id_train)]
y_test = Y[Y['client_id'].isin(id_test)]

In [ ]:
mx_train = pd.crosstab(x_train.client_id, x_train.title, normalize='index', values = x_train.user_score, aggfunc = np.sum)
class_train = mx_train.columns
mx_train = np.array(mx_train)

mx_test = pd.crosstab(x_test.client_id, x_test.title, normalize='index', values = x_test.user_score, aggfunc = np.sum)
mx_test = np.array(mx_test)

mx_train1 = pd.crosstab(x_train.client_id, x_train.hour, normalize='index')
mx_train1 = np.array(mx_train1)

mx_test1 = pd.crosstab(x_test.client_id, x_test.hour, normalize='index')
mx_test1 = np.array(mx_test1)

In [ ]:
import matplotlib.pyplot as plt
from sklearn import manifold
X_tsne = manifold.TSNE(n_components=2, init='pca', random_state=5, verbose=1).fit_transform(mx_train)

In [ ]:
#Data Visualization
x_min, x_max = X_tsne.min(0), X_tsne.max(0)
X_norm = (X_tsne - x_min) / (x_max - x_min)  #Normalize
plt.figure(figsize=(20, 20))
for i in range(X_norm.shape[0]):
    plt.text(X_norm[i, 0], X_norm[i, 1], s = i, size = 7)
plt.xticks([])
plt.yticks([])
plt.show()

# Test data

In [ ]:
my_train = pd.crosstab(y_train.client_id, y_train.title, normalize='index')
classy_train = list(my_train.columns)
my_train = np.array(my_train)

my_test = pd.crosstab(y_test.client_id, y_test.title, normalize='index')
classy_test = list(my_test.columns)
my_test = np.array(my_test)

#Model Building

In [ ]:
import numpy as np
import pandas as pd
import os
import warnings
from keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate, Reshape, BatchNormalization, LSTM, Reshape
from keras.models import Model
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
print(mx_train.shape)
print(mx_test.shape)
print(mx_train1.shape)
print(mx_test1.shape)
print(my_train.shape)
print(my_test.shape)

In [ ]:
def model_build():
  # Input Layer
  user_input = Input(shape=(mx_train.shape[1], ), name = "User-Input")
  user_vec = Dense(32, activation='relu')(user_input)

  pattern_input = Input(shape=(mx_train1.shape[1], ), name = "Pattern-Input")
  pattern_vec = Dense(8, activation='relu')(pattern_input)

  # Concatenate Features
  conc = Concatenate()([user_vec, pattern_vec])

  # Fully Connected Layer
  fc2 = BatchNormalization()(conc)
  fc3 = Dense(32, activation='relu')(fc2)
  fc4 = BatchNormalization()(fc3)
  fc5 = Dense(128, activation='relu')(fc4)
  fc6 = BatchNormalization()(fc5)

  # Output Layer
  out = Dense(my_train.shape[1], activation = 'softmax')(fc6)

  model = Model(inputs = [user_input, pattern_input], outputs = out)
  model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
  return model

In [ ]:
model = model_build()
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, show_shapes=True)

#Model Training

In [ ]:
import keras
callbacks = [keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)]
history = model.fit([mx_train, mx_train1], my_train, epochs=100, verbose=True, batch_size = 128, callbacks=callbacks, validation_split = 0.15)

#Performace Review

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
predictions = model.predict([mx_test, mx_test1])
pd.DataFrame(predictions, columns=classy_train)

In [ ]:
predictions = np.argmax(predictions, axis = 1)

In [ ]:
result = []
for i in predictions:
  result.append(classy_train[i])

In [ ]:
compare = pd.DataFrame()
compare['Predict'] = result
compare['Real'] = list(y_test['title'])
compare

In [ ]:
from sklearn.metrics import accuracy_score
print("預測準確率 %.2f"%(accuracy_score(result, list(y_test['title']))))

# LSTM model with Embedding Structure

In [ ]:
!pip install -q zhon

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
dataset = pd.read_csv('NewOCW.csv')
dataset = dataset.dropna()
dataset['client_id'] = dataset['client_id'].astype('str')
dataset['date'] = pd.to_datetime(dataset['date'])
dateset = dataset.sort_values(by='date', ascending=True, inplace=True)
dataset = dataset.drop(columns=['page', 'links'])

In [ ]:
import re
from zhon.hanzi import punctuation

user_mappings = {k:v for v, k in enumerate(dataset['client_id'].unique())}
dataset['client_id'] = dataset['client_id'].map(user_mappings)

punctuation_str = punctuation
dataset['title_tokenized'] = list(map(lambda x: re.sub("[%s]+" %punctuation_str, "", x), dataset['title']))
class_mappings = {k:v for v, k in enumerate(dataset['title_tokenized'].unique())}
dataset['title_tokenized'] = dataset['title_tokenized'].map(class_mappings)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

le=LabelEncoder()
dataset['title_encoded'] = le.fit_transform(dataset['title'])

In [ ]:
dataset.head(2)

In [ ]:
from random import sample
id = set(dataset['client_id'])
id_train = sample(id, int(len(id)*0.75))
id_test = []
for i in id:
  if i not in id_train:
    id_test.append(i)

print(len(id_train))
print(len(id_test))

In [ ]:
train = dataset[dataset['client_id'].isin(id_train)]
test = dataset[dataset['client_id'].isin(id_test)]

In [ ]:
# Use previous x-n behaviors to predict last n behaviors
from tqdm import tqdm

X_train = []
y_train = []
m = 75
for i in tqdm(set(train['client_id'])):
  temp = train[train['client_id'] == i]['title_tokenized'].values
  temp1 = train[train['client_id'] == i]['title_encoded'].values
  if len(temp) < m:
    continue
  temp = temp.reshape(-1, 1)
  temp1 = temp1.reshape(-1, 1)
  for j in range(m, temp.shape[0]):  
    X_train.append(temp[j-m:j, 0])
    y_train.append(temp1[j, 0].astype('float'))
X_train, y_train = np.array(X_train), np.array(y_train)

X_test = []
y_test = []
for i in tqdm(set(test['client_id'])):
  temp = test[test['client_id'] == i]['title_tokenized'].values
  temp1 = test[test['client_id'] == i]['title_encoded'].values
  if len(temp) < m:
    continue
  temp = temp.reshape(-1, 1)
  temp1 = temp1.reshape(-1, 1)
  for j in range(m, temp.shape[0]):  
    X_test.append(temp[j-m:j, 0])
    y_test.append(temp1[j, 0].astype('float'))
X_test, y_test = np.array(X_test), np.array(y_test)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
import numpy as np
import pandas as pd
import os
import warnings
from keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate, Reshape, BatchNormalization, LSTM, Reshape, Bidirectional, TimeDistributed
from keras.models import Model
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
embedding_size = len(set(y_train))

class_input = Input(shape=(X_train.shape[1], ), name = "Class-Input")
embedding_output = Embedding(input_dim = X_train.shape[1]+1, 
                                output_dim = embedding_size, 
                                name = "Behavior-Embedding")(class_input)
embedding = Model(class_input, embedding_output, name = 'Encoder')
embedding.summary()

In [ ]:
decoder_input = Input(shape=(m, embedding_size, ), name="decoder_input")
# LSTM Layer 
lstm = LSTM(units = 32, return_sequences = True)(decoder_input)
bn = BatchNormalization()(lstm)
lstm_1 = LSTM(units = 16, return_sequences = True)(bn)
bn1 = BatchNormalization()(lstm_1)
lstm_2 = LSTM(units = 8, return_sequences = False)(bn1)
bn2 = BatchNormalization()(lstm_2)

# Fully Connected Layer
fc3 = Dense(32, activation='relu')(bn2)
fc4 = BatchNormalization()(fc3)
fc5 = Dense(128, activation='relu')(fc4)
fc6 = BatchNormalization()(fc5)

# Output Layer
out = Dense(len(set(y_train)), activation = 'softmax')(fc6)

decoder = Model(decoder_input, out, name = 'Decoder')
decoder.summary()

In [ ]:
model_input = Input(shape=(X_train.shape[1], ))
embedding_output = embedding(model_input)
output = decoder(embedding_output)

model1 = Model(model_input, output)
model1.summary()

In [ ]:
model1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model1, show_shapes=True)

In [ ]:
import keras
callbacks = [keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)]
history = model1.fit(X_train, y_train, epochs=100, verbose=True, batch_size = 512, callbacks=callbacks, validation_split = 0.15)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
predictions = model1.predict(X_test)
predictions = np.argmax(predictions, axis = 1)
result = le.inverse_transform(predictions)

In [ ]:
compare = pd.DataFrame()
compare['Predict'] = result
compare['Real'] = le.inverse_transform(y_test.astype('int'))
compare

In [ ]:
from sklearn.metrics import accuracy_score
print("Prediction Accuracy %.2f"%(accuracy_score(result, le.inverse_transform(y_test.astype('int')))))

# Transformer model with position encoding

In [ ]:
import tensorflow as tf
import keras
from tensorflow.keras import regularizers
from tensorflow.keras import layers

In [ ]:
!pip install -q zhon

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
dataset = pd.read_csv('NewOCW.csv')
dataset = dataset.dropna()
dataset['client_id'] = dataset['client_id'].astype('str')
dataset['date'] = pd.to_datetime(dataset['date'])
dateset = dataset.sort_values(by='date', ascending=True, inplace=True)
dataset = dataset.drop(columns=['page', 'links'])

In [ ]:
import re
from zhon.hanzi import punctuation

user_mappings = {k:v for v, k in enumerate(dataset['client_id'].unique())}
dataset['client_id'] = dataset['client_id'].map(user_mappings)

punctuation_str = punctuation
dataset['title_tokenized'] = list(map(lambda x: re.sub("[%s]+" %punctuation_str, "", x), dataset['title']))
class_mappings = {k:v for v, k in enumerate(dataset['title_tokenized'].unique())}
dataset['title_tokenized'] = dataset['title_tokenized'].map(class_mappings)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

le=LabelEncoder()
dataset['title_encoded'] = le.fit_transform(dataset['title'])

In [ ]:
dataset.head(2)

In [ ]:
from random import sample
id = set(dataset['client_id'])
id_train = sample(id, int(len(id)*0.75))
id_test = []
for i in id:
  if i not in id_train:
    id_test.append(i)

print(len(id_train))
print(len(id_test))

In [ ]:
train = dataset[dataset['client_id'].isin(id_train)]
test = dataset[dataset['client_id'].isin(id_test)]

In [ ]:
# Use previous x-n behaviors to predict last n behaviors
from tqdm import tqdm

X_train = []
y_train = []
m = 75
for i in tqdm(set(train['client_id'])):
  temp = train[train['client_id'] == i]['title_tokenized'].values
  temp1 = train[train['client_id'] == i]['title_encoded'].values
  if len(temp) < m:
    continue
  temp = temp.reshape(-1, 1)
  temp1 = temp1.reshape(-1, 1)
  for j in range(m, temp.shape[0]):  
    X_train.append(temp[j-m:j, 0])
    y_train.append(temp1[j, 0].astype('float'))
X_train, y_train = np.array(X_train), np.array(y_train)

X_test = []
y_test = []
for i in tqdm(set(test['client_id'])):
  temp = test[test['client_id'] == i]['title_tokenized'].values
  temp1 = test[test['client_id'] == i]['title_encoded'].values
  if len(temp) < m:
    continue
  temp = temp.reshape(-1, 1)
  temp1 = temp1.reshape(-1, 1)
  for j in range(m, temp.shape[0]):  
    X_test.append(temp[j-m:j, 0])
    y_test.append(temp1[j, 0].astype('float'))
X_test, y_test = np.array(X_test), np.array(y_test)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [ ]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [ ]:
embed_dim = len(set(y_train))  # Embedding size for each token
num_heads = 10  # Number of attention heads
ff_dim = 256  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(X_train.shape[1],))
embedding_layer = TokenAndPositionEmbedding(X_train.shape[1], len(set(y_train)), embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(len(set(y_train))*2, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
outputs = layers.Dense(len(set(y_train)), activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.summary()

In [ ]:
import keras
from keras.callbacks import ReduceLROnPlateau
red_lr= ReduceLROnPlateau(monitor='val_accuracy',patience=3,verbose=0,factor=0.1)
callbacks = [keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)]

history = model.fit(X_train, y_train, epochs=100, verbose=True, batch_size = 512, callbacks=[red_lr, callbacks], validation_split = 0.15)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
predictions = model.predict(X_test)
predictions = np.argmax(predictions, axis = 1)
result = le.inverse_transform(predictions)

In [ ]:
compare = pd.DataFrame()
compare['Predict'] = result
compare['Real'] = le.inverse_transform(y_test.astype('int'))
compare

In [ ]:
from sklearn.metrics import accuracy_score
print("Prediction Accuracy %.2f"%(accuracy_score(result, le.inverse_transform(y_test.astype('int')))))

# Autoencoder for Segmentation

In [ ]:
dataset = pd.read_csv('NewOCW.csv')
dataset = dataset.dropna()
dataset['client_id'] = dataset['client_id'].astype('str')
dataset['date'] = pd.to_datetime(dataset['date'])
dateset = dataset.sort_values(by='date', ascending=True, inplace=True)
dataset = dataset.drop(columns=['page', 'links'])

In [ ]:
user_mappings = {k:v for v, k in enumerate(dataset['client_id'].unique())}
dataset['client_id'] = dataset['client_id'].map(user_mappings)

In [ ]:
dataset['hour'] = list(map(lambda x: str(x)[11:13], dataset['date']))
cc = dataset.groupby('client_id')['title'].nunique()
cc = pd.DataFrame(cc)
cc = cc.rename(columns={'title': 'number'})
dataset = pd.merge(dataset, cc, on="client_id")
del cc

In [ ]:
import time
from datetime import datetime

dataset['time_delta'] = (max(dataset['date']) - dataset['date'])
dataset['time_delta'] = list(map(lambda x: x.days, dataset['time_delta']))

In [ ]:
r = 0.8
dataset['user_score'] = r**dataset['time_delta']

In [ ]:
from random import sample
id = list(dataset['client_id'].unique())
id_train = sample(list(id), int(len(id)*0.75))
id_test = []
for i in id:
  if i not in id_train:
    id_test.append(i)

In [ ]:
x_train = dataset[dataset['client_id'].isin(id_train)]
x_test = dataset[dataset['client_id'].isin(id_test)]

In [ ]:
mx_train = pd.crosstab(x_train.client_id, x_train.title, normalize='index', values = x_train.user_score, aggfunc = np.sum)
mx_train = np.array(mx_train)

mx_test = pd.crosstab(x_test.client_id, x_test.title, normalize='index', values = x_test.user_score, aggfunc = np.sum)
mx_test = np.array(mx_test)

In [ ]:
mx_train1 = pd.crosstab(x_train.client_id, x_train.hour, normalize='index')
mx_train1 = np.array(mx_train1)

mx_test1 = pd.crosstab(x_test.client_id, x_test.hour, normalize='index')
mx_test1 = np.array(mx_test1)

In [ ]:
train = np.concatenate([mx_train, mx_train1], axis = 1)
test = np.concatenate([mx_test, mx_test1], axis = 1)

print(train.shape)
print(test.shape)

In [ ]:
import numpy as np
import pandas as pd
import os
import warnings
from keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate, Reshape, BatchNormalization, LSTM, Reshape
from keras.models import Model
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
encoding_dim = 3

user_input = Input(shape=(train.shape[1], ), name = "User-Input")
x = Dense(128, activation='relu')(user_input)
x = BatchNormalization()(x)
x = Dense(64, activation='relu')(x)
x = BatchNormalization()(x)
x = Dense(32, activation='relu')(x)
x = BatchNormalization()(x)
x = Dense(16, activation='relu')(x)
x = BatchNormalization()(x)
x = Dense(8, activation='relu')(x)
x = BatchNormalization()(x)
x = Dense(4, activation='relu')(x)
x = BatchNormalization()(x)
encoded_output = Dense(encoding_dim, activation='relu')(x)

In [ ]:
encoder = Model(user_input, encoded_output, name = 'Encoder')
encoder.summary()

In [ ]:
decoder_input = Input(shape=(encoding_dim, ), name="decoder_input")
x = Dense(4, activation='relu')(decoder_input)
x = BatchNormalization()(x)
x = Dense(8, activation='relu')(x)
x = BatchNormalization()(x)
x = Dense(16, activation='relu')(x)
x = BatchNormalization()(x)
x = Dense(32, activation='relu')(x)
x = BatchNormalization()(x)
x = Dense(64, activation='relu')(x)
x = BatchNormalization()(x)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
decoded_output = Dense(train.shape[1], activation='relu')(x)

In [ ]:
decoder = Model(decoder_input, decoded_output, name = 'Decoder')
decoder.summary()

In [ ]:
ae_input = Input(shape=(train.shape[1], ), name="AE_input")
ae_encoder_output = encoder(ae_input)
ae_decoder_output = decoder(ae_encoder_output)

In [ ]:
ae = Model(ae_input, ae_decoder_output, name="AE")
ae.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(ae, show_shapes=True)

In [ ]:
ae.compile(optimizer='adam', loss='mse', metrics = ['mae'])

In [ ]:
import keras
callbacks = [keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)]

History = ae.fit(train, train, epochs=100, batch_size=128, shuffle=True, validation_split = 0.1, callbacks=callbacks)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(History.history['loss'])
plt.plot(History.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
ae.evaluate(test, test)

In [ ]:
user = np.concatenate([train, test], axis = 0)
encoded_data = encoder.predict(user)
decoded_data = decoder.predict(encoded_data)

In [ ]:
columns_names = []
for i in range(encoding_dim):
  columns_names.append('dim_'+str(i+1))
result = pd.DataFrame(encoded_data, columns = columns_names)

In [ ]:
result

In [ ]:
plt.figure(figsize=(10, 10)) 
plt.scatter(result["dim_1"], result["dim_2"], s = 5)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d.axes3d import Axes3D

fig = plt.figure()

ax = Axes3D(fig)
ax.scatter(result["dim_1"], result["dim_2"], result['dim_3'], c ='blue', s = 1)

plt.show()

In [ ]:
from sklearn import metrics
from sklearn.cluster import KMeans
from tqdm import tqdm

clusters = range(2,50)
sc_scores = []
for k in tqdm(clusters):
  kmeans_model = KMeans(n_clusters=k).fit(result)
  sc_score = metrics.silhouette_score(result, kmeans_model.labels_,sample_size=10000, metric='euclidean')
  sc_scores.append(sc_score)
print(sc_scores)
best_clusters = sc_scores.index(max(sc_scores))+2
print(best_clusters)

In [ ]:
plt.figure()
plt.plot(clusters, sc_scores, )
plt.xlabel('Clusters',fontsize=18)
plt.ylabel('Silhouette Coefficient Score',fontsize=18)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.show()

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=best_clusters)
kmeans.fit(result)

plt.rcParams['font.size'] = 14
plt.figure(figsize=(10, 10))
plt.title('Segmented data')
plt.scatter(result['dim_1'], result['dim_2'], c = kmeans.predict(result), s = 5)

In [ ]:
fig = plt.figure()

ax = Axes3D(fig)
ax.scatter(result["dim_1"], result["dim_2"], result['dim_3'], c = kmeans.predict(result), s = 1)

plt.show()